# ДЗ 4

In [1]:
!pip install implicit

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -R  /content/drive/MyDrive/Recomendation/srcrec  /

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

base_path = "/content/drive/MyDrive/Recomendation/"

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from srcrec.metrics import precision_at_k, recall_at_k
from srcrec.utils import prefilter_items

In [5]:
from srcrec.recommenders import MainRecommender

In [6]:
data = pd.read_csv(base_path + 'retail_train.csv') 

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] <  data['week_no'].max() - test_size_weeks]
data_test  = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
recommender = MainRecommender(data_train)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


---

### get_similar_items_recommendation

In [9]:
popularity = recommender.prep_data.loc[recommender.prep_data["user_id"] == 2233, ['item_id', 'quantity']].\
                groupby(['item_id'])['quantity'].count().reset_index()
popularity.sort_values('quantity', ascending=False, inplace=True)
top_item_list = popularity.loc[popularity["item_id"] != 999999, "item_id"][:5].tolist()
top_item_list

[6534178, 1058997, 1082185, 1081177, 828935]

In [10]:
rez = set([recommender.id_to_itemid[recommender.model.similar_items(recommender.itemid_to_id[item], N = 1)[0][0]] for item in top_item_list])
rez

{828935, 1058997, 1081177, 1082185, 6534178}

In [11]:
recommender.get_similar_items_recommendation(user=2233)

{828935, 1058997, 1081177, 1082185, 6534178}

---

### get_similar_users_recommendation

In [12]:
user_id = 2468

In [13]:
sim_users_list = [recommender.id_to_userid[usr] for usr, p in recommender.model.similar_users(recommender.userid_to_id[user_id], 
                                                                                              N=5)]
sim_users_list

[2468, 145, 321, 258, 409]

In [14]:
sim_users_list = [recommender.id_to_userid[usr] for usr, p in recommender.model.similar_users(recommender.userid_to_id[user_id], 
                                                                                              N=5+1)[1:5+1]]
sim_users_list

[145, 321, 258, 409, 2109]

In [15]:
rez = set([list(recommender.get_recommendations(user, rec_num=1))[0] for user in sim_users_list])
rez

{873874, 1040371, 1068708, 7409971, 14043826}

In [16]:
recommender.get_similar_users_recommendation(user_id)

{873874, 1040371, 1068708, 7409971, 14043826}